In [1]:
from timeit import default_timer as timer

start = timer()

In [2]:
import os
import cupy as cp
import numpy as np

os.environ["GROGUPY_ARCHITECTURE"] = "GPU"
os.environ["GROGUPY_PARALLEL_SIZE"] = str(cp.cuda.runtime.getDeviceCount())
import grogu
import pickle
import sisl
from concurrent.futures import ThreadPoolExecutor

/home/c_magpd/.local/lib/python3.10/site-packages/grogu/_tqdm.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as _tqdm


In [3]:
grogu._environ.GROGUPY_GET_ENVIRONMENT_VARIABLE("GROGUPY_PARALLEL_SIZE")

1

In [4]:
infile = "/home/c_magpd/c_magex/dani/grogu_testing/lat3_791/Fe3GeTe2.fdf"

In [5]:
kspace = grogu.Kspace(
    "xy",
    100,
)
contour = grogu.Contour(
    None,
    600,
    1000,
    eigfile=infile,
)
hamiltonian = grogu.Hamiltonian(
    infile,
    [0, 0, 1],
)
simulation = grogu.Builder(np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]))
simulation.add_kspace(kspace)
simulation.add_contour(contour)
simulation.add_hamiltonian(hamiltonian)
magnetic_entities = [
    dict(atom=3, l=2),
    dict(atom=4, l=2),
    dict(atom=5, l=2),
]
pairs = [
    dict(ai=0, aj=1, Ruc=np.array([0, 0, 0])),
    dict(ai=0, aj=2, Ruc=np.array([0, 0, 0])),
    dict(ai=1, aj=2, Ruc=np.array([0, 0, 0])),
    dict(ai=0, aj=0, Ruc=np.array([1, 0, 0])),
    dict(ai=0, aj=1, Ruc=np.array([1, 0, 0])),
    dict(ai=0, aj=2, Ruc=np.array([1, 0, 0])),
    dict(ai=0, aj=0, Ruc=np.array([2, 0, 0])),
]
magnetic_entities = simulation.create_magnetic_entities(magnetic_entities)
simulation.add_magnetic_entities(magnetic_entities)
pairs = simulation.create_pairs(pairs)
simulation.add_pairs(pairs)

simulation.finalize()
print((timer() - start) / 60, " min")
print(simulation)

Spin box Hamiltonian:   0%|          | 0/81 [00:00<?, ?it/s]

Spin box Overlap matrix:   0%|          | 0/81 [00:00<?, ?it/s]

Symmetrize Hamiltonian:   0%|          | 0/81 [00:00<?, ?it/s]

Transpose Hamiltonian:   0%|          | 0/81 [00:00<?, ?it/s]

Spin trace H_TRB:   0%|          | 0/81 [00:00<?, ?it/s]

...

Add pairs:   0%|          | 0/7 [00:00<?, ?it/s]

Rotating Exchange field: 0it [00:00, ?it/s]

Rotating Exchange field: 0it [00:00, ?it/s]

Rotating Exchange field: 0it [00:00, ?it/s]

0.08846067298436537  min
grogu version: 1.0.0
Input file: /home/c_magpd/c_magex/dani/grogu_testing/lat3_791/Fe3GeTe2.fdf
SLURM job ID: 9858751
Architecture: GPU
Number of GPUs in the cluster: 1
Solver used for Exchange tensor: Fit
Solver used for Anisotropy tensor: Fit
Cell [Ang]:
3.791001511088653242e+00 0.000000000000000000e+00 0.000000000000000000e+00
-1.895500755544326621e+00 3.283103614407953064e+00 0.000000000000000000e+00
0.000000000000000000e+00 0.000000000000000000e+00 2.057000819825037041e+01

DFT axis: [0 0 1]
Quantization axis and perpendicular rotation directions:
[1. 0. 0.] --> [array([ 0.,  0., -1.]), array([0., 1., 0.])]
[0. 1. 0.] --> [array([1., 0., 0.]), array([ 0.,  0., -1.])]
[0. 0. 1.] --> [array([1., 0., 0.]), array([0., 1., 0.])]
Parameters for the contour integral:
Number of k points: 100
k point directions: xy
Ebot: -12.806878959999999        WARNING: This was automatically determined!
Etop: 0
Eset: 600
Esetp: 1000



In [ ]:
num_cpu = 4
builder = simulation
_tqdm = grogu._tqdm._tqdm

# iterate over the reference directions (quantization axes)
G_mag = []
G_pair_ij = []
G_pair_ji = []

for orient in builder.ref_xcf_orientations:
    G_mag.append([])
    G_pair_ij.append([])
    G_pair_ji.append([])

    # obtain rotated exchange field and Hamiltonian
    for mag_ent in builder.magnetic_entities:
        G_mag[-1].append(
            cp.zeros(
                (builder.contour.eset, mag_ent.SBS, mag_ent.SBS),
                dtype="complex128",
            )
        )

    for pair in builder.pairs:
        G_pair_ij[-1].append(
            cp.zeros((builder.contour.eset, pair.SBS1, pair.SBS2), dtype="complex128")
        )
        G_pair_ji[-1].append(
            cp.zeros((builder.contour.eset, pair.SBS2, pair.SBS1), dtype="complex128")
        )
G_mag = cp.array(G_mag)
G_pair_ij = cp.array(G_pair_ij)
G_pair_ji = cp.array(G_pair_ji)

SBI = cp.array([m.spin_box_indices for m in builder.magnetic_entities])
SBI1 = cp.array([p.SBI1 for p in builder.pairs])
SBI2 = cp.array([p.SBI2 for p in builder.pairs])
Ruc = cp.array([p.supercell_shift for p in builder.pairs])

sc_off = cp.array(builder._dh.sc_off)
eset = builder.contour.eset


rotated_H = [H.H for H in builder._rotated_hamiltonians]
rotated_S = [H.S for H in builder._rotated_hamiltonians]

all_kpoints = np.array_split(builder.kspace.kpoints, num_cpu)
all_kweights = np.array_split(builder.kspace.weights, num_cpu)
samples = cp.array(builder.contour.samples.reshape(eset, 1, 1))
print((timer() - start) / 60, " min")

from concurrent.futures import ThreadPoolExecutor


def process_gpu(gpu_number):
    with cp.cuda.Device(gpu_number):
        local_kpoints = all_kpoints[gpu_number]
        local_kweights = all_kweights[gpu_number]
        local_kpoints = cp.array(local_kpoints)
        local_kweights = cp.array(local_kweights)
        local_SBI = cp.array(SBI)
        local_SBI1 = cp.array(SBI1)
        local_SBI2 = cp.array(SBI2)
        local_Ruc = cp.array(Ruc)

        local_sc_off = cp.array(sc_off)
        local_samples = cp.array(samples)

        local_G_mag = cp.zeros_like(G_mag)
        local_G_pair_ij = cp.zeros_like(G_pair_ij)
        local_G_pair_ji = cp.zeros_like(G_pair_ji)

        for i in _tqdm(range(len(local_kpoints))):
            # weight of k point in BZ integral
            wk = local_kweights[i]
            k = local_kpoints[i]

            # iterate over reference directions
            for j in range(len(rotated_H)):
                # calculate Hamiltonian and Overlap matrix in a given k point
                # this generates the list of phases
                phases = cp.exp(-1j * 2 * cp.pi * k @ local_sc_off.T)

                # phases applied to the hamiltonian
                HK = cp.einsum("abc,a->bc", cp.array(rotated_H[j]), phases)
                SK = cp.einsum("abc,a->bc", cp.array(rotated_S[j]), phases)

                Gk = cp.linalg.inv(SK * local_samples - HK)

                # store the Greens function slice of the magnetic entities
                for l, sbi in enumerate(local_SBI):
                    local_G_mag[j][l] = Gk[..., sbi, :][..., sbi] * wk

                for l, dat in enumerate(zip(local_SBI1, local_SBI2, local_Ruc)):
                    sbi1, sbi2, ruc = dat
                    phase = cp.exp(1j * 2 * cp.pi * k @ ruc.T)

                    # store the Greens function slice of the magnetic entities
                    local_G_pair_ij[j][l] += Gk[..., sbi1, :][..., sbi2] * wk * phase
                    local_G_pair_ji[j][l] += Gk[..., sbi2, :][..., sbi1] * wk / phase

    return local_G_mag, local_G_pair_ij, local_G_pair_ji


asd = timer()
# Replace the GPU loop with this:
with ThreadPoolExecutor(max_workers=num_cpu) as executor:
    futures = [executor.submit(process_gpu, gpu_id) for gpu_id in range(num_cpu)]
    results = [future.result() for future in futures]
print((timer() - asd) / 60, " min")

# Combine results
G_mag = cp.asnumpy(G_mag)
G_pair_ji = cp.asnumpy(G_pair_ji)
G_pair_ij = cp.asnumpy(G_pair_ij)
for G_mag_local, G_pair_ij_local, G_pair_ji_local in results:
    G_mag += cp.asnumpy(G_mag_local)
    G_pair_ij += cp.asnumpy(G_pair_ij_local)
    G_pair_ji += cp.asnumpy(G_pair_ji_local)

print((timer() - asd) / 60, " min")

# sum reduce partial results of mpi nodes
for j in range(len(builder._rotated_hamiltonians)):
    for l in range(len(SBI)):
        mag_ent = builder.magnetic_entities[l]
        mag_ent.Gii[j] = G_mag[j][l]

        mag_ent.calculate_energies(builder.contour.weights)
        mag_ent.fit_anisotropy_tensor(builder.ref_xcf_orientations)

    for l in range(len(SBI1)):
        pair = builder.pairs[l]
        pair.Gij[j] = G_pair_ij[j][l]
        pair.Gji[j] = G_pair_ji[j][l]

        pair.calculate_energies(builder.contour.weights)
        pair.fit_exchange_tensor(builder.ref_xcf_orientations)

print("Runtime:", (timer() - asd), "s")

0.09313269895113384  min


  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
print([print(p.J_iso_meV) for p in builder.pairs])
print([print(m.K_meV[0, 0]) for m in builder.magnetic_entities])

In [ ]:
stop = timer()
print("Runtime:", (stop - start) / 60, "min")